In [1]:
import numpy as np

import multiprocessing as mp

In [2]:
# Corollary 2.4 in Mohammadi 2014 - for multi-d
def alpha_estimator(m, X):
    # X is N by d matrix
    N = len(X)
    n = int(N/m) # must be an integer
    Y = np.sum(X.reshape((n, m, -1)), 1)
    eps = np.spacing(1)
    Y_log_norm = np.log(np.linalg.norm(Y, axis=1) + eps).mean()
    X_log_norm = np.log(np.linalg.norm(X, axis=1) + eps).mean()
    diff = (Y_log_norm - X_log_norm) / np.log(m)
    return 1/diff

In [3]:
# number of data points
n = 10000
# dimension of x
d = 100
# batch size
batch = 10

sig_w = 3
sig_x = 1
sig_y = 3

# std_noise = 3
w = np.random.normal(0,3,d)
# w = sig_w * np.random.randn(d,1)
X = sig_x * np.random.randn(n,d)
Y = X@w.reshape(-1,1) + sig_y * np.random.randn(n,1)

In [42]:
w

array([-1.45406046, -0.80093886,  0.83098613,  1.82603291, -1.24200221,
       -0.3946757 , -0.14342238, -2.42918843,  0.99493286,  1.08779443,
       -1.84400102, -3.31681272, -2.08754629,  0.89727946,  4.25631219,
        0.08290963,  2.64297729,  1.78057525,  2.4809494 , -6.49470891,
        0.59788779,  1.38078902,  5.30663381, -0.654131  ,  3.25096209,
        5.13656815,  0.28275396, -1.51822524, -4.17694677, -5.37275743,
       -7.50297622,  3.24732353,  4.17139494,  0.85567767,  0.87073298,
       -0.17368911,  4.39234183, -1.78879271,  1.38368976, -2.15050996,
        0.1175499 , -3.27404807,  2.61296333, -0.0979814 , -0.18741849,
       -1.29345061, -2.43955255,  1.5521806 ,  0.86962613,  1.09629518,
       -3.5042196 , -3.34261522,  2.50711399, -0.43168249, -0.51970669,
       -5.18334007, -1.45891713,  0.04901316, -4.79320009, -0.23522375,
        6.18717488, -2.03054342, -1.26570271, -1.61952501,  2.19106975,
       -2.70225693, -0.69967819, -4.09411861, -2.50394578, -8.55

In [4]:
def loss_batch(w,X,Y,batch):
    idx = np.random.randint(Y.shape[0], size = batch)
    X1 = X[idx,:]
    Y1 = Y[idx,:]
    loss = np.sum(np.square(X1@w -Y1))/(2*batch)
    return loss

def loss(w,X,Y):
    loss = np.sum(np.square(np.dot(X,w) -Y))/(2*X.shape[0])
    return loss

def linearreg(d,X,Y,K,stepsize,batch,w_star):
    loss_list = []
    w_list = []
    
    w0 = np.random.uniform(-3,3,d)
    w_list.append(w0)
    loss_list.append(loss_batch(w0,X,Y,batch))
    
    for k in range(K):
        
        idx = np.random.randint(Y.shape[0], size = batch)
        Xk = X[idx,:]
        Yk = Y[idx,:]
        w = w_list[-1].reshape(-1,1) - stepsize / batch * (Xk.T @ (Xk @ w_list[-1].reshape(-1,1) - Yk))
        w_list.append(w.reshape(-1))
        
        loss_list.append(loss_batch(w,X,Y,batch))
        
    w_list = np.array(w_list)
    loss_list = np.array(loss_list)
    
    w_mean = np.mean(w_list[-500:], axis = 0)
    
    return w_mean

def linearreg_unif(d,X,Y,K,base_lr,max_lr,batch,w_star):
    loss_list = []
    w_list = []
    
    w0 =np.random.uniform(-3,3,d)
    w_list.append(w0)
    loss_list.append(loss_batch(w0,X,Y,batch))
    
    for k in range(K):
        
        stepsize = np.random.uniform(low=base_lr, high=max_lr)
        idx = np.random.randint(Y.shape[0], size = batch)
        Xk = X[idx,:]
        Yk = Y[idx,:]
#         temp = Xk @ w_list[-1].reshape(-1,1) - Yk
#         temp = Xk.T @ temp
#         print(temp.shape)
        
        w = w_list[-1].reshape(-1,1) - stepsize / batch * (Xk.T @ (Xk @ w_list[-1].reshape(-1,1) - Yk))
#         print((Xk @ w_list[-1].reshape(-1,1) )shape)
        w_list.append(w.reshape(-1))
        
        loss_list.append(loss_batch(w,X,Y,batch))
        
    w_list = np.array(w_list)
    loss_list = np.array(loss_list)
    
    w_mean = np.mean(w_list[-500:], axis = 0)
    w_final = w_list[-1] - w_star
    
    return w_mean, w_final, loss_list

def linearreg_cyc(d,X,Y,K,base_lr,max_lr,stepsize,batch,w_star):
    loss_list = []
    w_list = []
    
    w0 =np.random.uniform(-3,3,d)
    w_list.append(w0)
    loss_list.append(loss_batch(w0,X,Y,batch))
    
    for k in range(K):
        
        cycle = np.floor(1 + (k+1) / (2*stepsize))
        loc = np.abs((k+1) / stepsize - 2*cycle +1)
        lr = base_lr + (max_lr - base_lr) * max(0,(1-loc))
        
        idx = np.random.randint(Y.shape[0], size = batch)
        Xk = X[idx,:]
        Yk = Y[idx,:]
        w = w_list[-1].reshape(-1,1) - stepsize / batch * (Xk.T @ (Xk @ w_list[-1].reshape(-1,1) - Yk))
        w_list.append(w.reshape(-1))
        
        loss_list.append(loss_batch(w,X,Y,batch))
        
    w_list = np.array(w_list)
    loss_list = np.array(loss_list)
    
    w_mean = np.mean(w_list[-500:], axis = 0)
    w_final = w_list[-1] - w_star
    
    return w_mean, w_final

def linearreg_mc(d,X,Y,K,base_lr,max_lr,p,stepsize,batch,w_star):
    loss_list = []
    w_list = []
    
    onestep = (max_lr-base_lr)/stepsize
    
    w0 = np.random.uniform(-3,3,d)
    w_list.append(w0)
    loss_list.append(loss_batch(w0,X,Y,batch))
#     lr_list=[]
    
    for k in range(K):
        
        if k == 0:
            lr = base_lr
        elif k == 1:
            lr = base_lr + onestep
        else:
            if lr <= base_lr:
                p = max(p, 1-p)
                lr = base_lr + onestep
            elif lr >= max_lr:
                p = min(p,1-p)
                lr = max_lr - onestep
            else:
                temp = np.random.uniform(0,1)
                if temp <= p:
                    lr = min(max_lr, lr+onestep)
                else:
                    lr = max(base_lr, lr-onestep)
#         lr_list.append(lr)
        idx = np.random.randint(Y.shape[0], size = batch)
        Xk = X[idx,:]
        Yk = Y[idx,:]
        w = w_list[-1].reshape(-1,1) - stepsize / batch * (Xk.T @ (Xk @ w_list[-1].reshape(-1,1) - Yk))
        w_list.append(w.reshape(-1))
        
        loss_list.append(loss_batch(w,X,Y,batch))
        
    w_list = np.array(w_list)
    loss_list = np.array(loss_list)
    
    w_mean = np.mean(w_list[-500:], axis = 0)
    w_final = w_list[-1] - w_star
    
    return w_mean, w_final#, lr_list

def ideal_sol(X,Y):
    temp = np.dot(np.transpose(X), X)
    temp = np.linalg.inv(temp)
    temp = np.dot(temp, np.transpose(X))
    temp = np.dot(temp, Y)
    return temp

In [7]:
#  lower constant lr

mean_lr = [0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22]
delta = 0.05
d = 100
K = 1000
T = 10000
stepsize = 0.02
batch = 10
alpha_list = []
w_star = ideal_sol(X,Y)

# pool = mp.Pool(mp.cpu_count())

for mean in mean_lr:
    random_vec = []
    
    for t in range(T):
        diff = linearreg(d, X, Y, K, mean-delta, batch, w_star)
#         diff = w - w_star
        random_vec.append(diff)
#         diff_list.append(diff)
    
#     random_vec = [pool.apply(linearreg_unif, args=(1,X,Y,K,base,max_lr,batch,w_star)) for t in range(T)]
    
    
    random_vec = np.array(random_vec)
    data = np.array(random_vec - np.mean(random_vec, axis = 0))
    alpha_list.append(alpha_estimator(100,data))
    
# pool.close()

In [8]:
alpha_list

[1.99583675026005,
 2.0122145594803498,
 1.996048906029783,
 1.9950367059264706,
 2.004977513702677,
 1.9973217012957465,
 2.0072649318091154,
 2.0039254246749505,
 2.0048269735745183]

In [9]:
#  upper constant lr

mean_lr = [0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22]
delta = 0.05
d = 100
K = 1000
T = 10000
stepsize = 0.02
batch = 10
alpha_list = []
w_star = ideal_sol(X,Y)

# pool = mp.Pool(mp.cpu_count())

for mean in mean_lr:
    random_vec = []
    
    for t in range(T):
        diff = linearreg(d, X, Y, K, mean+delta, batch, w_star)
#         diff = w - w_star
        random_vec.append(diff)
#         diff_list.append(diff)
    
#     random_vec = [pool.apply(linearreg_unif, args=(1,X,Y,K,base,max_lr,batch,w_star)) for t in range(T)]
    
    
    random_vec = np.array(random_vec)
    data = np.array(random_vec - np.mean(random_vec, axis = 0))
    alpha_list.append(alpha_estimator(100,data))
    
# pool.close()

In [10]:
alpha_list

[1.503881464560734,
 1.3665886277761965,
 1.2650602704559242,
 1.1628364962942015,
 1.047825392474475,
 0.9985003764905789,
 0.9939563427605188,
 0.9236492483200796,
 0.9019350759233753]

In [54]:
alpha_list

[2.0076225726704817,
 1.993290836651396,
 2.009862545076988,
 2.0058438625085917,
 1.831623996601687,
 1.470831536025423,
 1.3533148198418132,
 1.2454126140725206,
 1.1417129926018952]